# Tutorial 3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd #use to manipulate data
import matplotlib.pyplot as plt #use for interactive visualizations
import cv2 #display image
import numpy as np #use numpy library for matrixes, arrays
import seaborn as sns # Statistics plotting

from pathlib import Path

In [ ]:
cwd = Path("/content/drive/MyDrive/NICERlab/sample_results_mapping")
data_dir = Path("/content/drive/MyDrive/SVI_key1_labeled_data/")

! ls {data_dir}

In [ ]:
videos = list(data_dir.glob('*/videos/*key_1.mp4'))
dlc_files = list(data_dir.glob('*/videos/*.h5'))

data = list(zip(videos, dlc_files))

In [ ]:
df = pd.DataFrame(data, columns=['video', 'dlc_file'])
df

In [ ]:
def get_name(path):
  filename = path.parts[-1]
  return filename[:4]

df["name"] = df.video.apply(get_name)
df

## Load data

In [ ]:
def load_data(dlc_file):
  keypoints = pd.read_hdf(dlc_file) #identify key regions of an object that are used as the base to later match and identify it in a new image
  return keypoints

load_data(df.iloc[0].dlc_file)

## Visualizations

In [ ]:
gs = plt.GridSpec(1, 2)

plt.subplot(gs[0])
keypoints = load_data(df.iloc[0].dlc_file)
pinky_tips = keypoints.xs("right index tip", level='bodyparts', axis=1).droplevel("scorer", axis=1)
plt.scatter(pinky_tips.x, pinky_tips.y)

plt.subplot(gs[1])
keypoints = load_data(df.iloc[0].dlc_file)
pinky_tips = keypoints.xs("left index tip", level='bodyparts', axis=1).droplevel("scorer", axis=1)
plt.scatter(pinky_tips.x, pinky_tips.y)

In [ ]:
class VideoReader:
  def __init__(self, video):
    self.video = str(video)
    self.pos = 0
    self.cap = cv2.VideoCapture(self.video)
    self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    self.fps = float(self.cap.get(cv2.CAP_PROP_FPS))

  def __len__(self):
    return int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

  def __getitem__(self, idx):
    if self.pos != idx:
      self.cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
      self.pos = idx

    ret, frame = self.cap.read()
    frame = np.flip(frame, axis=2)
    self.pos += 1
    return frame

In [ ]:
ncols = 7
nrows = 1

for idx, row in df.iterrows():
  plt.figure(figsize=(26, 8))
  gs = plt.GridSpec(nrows=nrows, ncols=ncols)

  keypoints = load_data(row.dlc_file)
  likelihood = keypoints.xs("likelihood", level="coords", axis=1).droplevel(0, axis=1)
  keypoints = keypoints.where(likelihood > 0.6)
  # keypoints.interpolate
  # keypoints = keypoints.iloc[200:400] # video trimming

  cap = cv2.VideoCapture(str(row.video))
  reader = VideoReader(row.video)
  cap.set(cv2.CAP_PROP_POS_FRAMES, 300) #capture property position of frame #change back to 200 if doesnt work
  ret, frame = cap.read()
  frame = np.flip(frame, axis=2) # BGR -> RGB color swap #3 layers exist in matrix(video), axis 2 is 3rd layer which is where colours lie

  plt.subplot(gs[0])
  plt.ylabel(row["name"])
  plt.imshow(frame)

#for right hand
  plt.subplot(gs[1])
  bodypart_df = keypoints.xs("right index tip", level='bodyparts', axis=1).droplevel("scorer", axis=1)
  c = np.arange(keypoints.shape[0]) # COLOUR oldest to newest movement
  plt.scatter(bodypart_df.x, bodypart_df.y, c=c)
  plt.imshow(frame)

  plt.subplot(gs[3])
  ax = plt.gca() #get current axis (subplot)
  deltas = bodypart_df.diff()
  distances = np.linalg.norm(deltas[["x", "y"]], axis=1)
  distances = np.sqrt(deltas.x**2 + deltas.y**2)
  sns.histplot(distances, ax=ax, bins=np.arange(20))

  plt.subplot(gs[5])
  from tqdm import tqdm
  from scipy.stats import gaussian_kde

  # make these smaller to increase the resolution
  dx, dy = 2, 2

  # generate 2 2d grids for the x & y bounds
  w = reader.width
  h = reader.height
  result = {}

  # Get the centroid of the bodypart by averaging the bodypart positions in each frame
  data = bodypart_df[['y', 'x']].dropna().values.T

  # Get the density of time spent in location (down sampled for 1 frame every 100)
  k = gaussian_kde(data[:,::1], )
  mgrid = np.mgrid[:h, :w]
  z = k(mgrid.reshape(2, -1))


  plt.title("Box Reference")
  plt.imshow(reader[0])

  plt.title("Right Hand")
  plt.imshow(frame) # plotting a frame sets up the matplotlib axis correctly
  plt.imshow(z.reshape(mgrid.shape[1:]))
  plt.contourf(z.reshape(mgrid.shape[1:]), cmap='seismic', alpha=1, levels=20)


#for left hand
  plt.subplot(gs[2])
  bodypart_df = keypoints.xs("left index tip", level='bodyparts', axis=1).droplevel("scorer", axis=1)
  c = np.arange(keypoints.shape[0])
  plt.scatter(bodypart_df.x, bodypart_df.y, c=c)
  plt.imshow(frame)

  plt.subplot(gs[4])
  ax = plt.gca()
  deltas = bodypart_df.diff()
  distances = np.linalg.norm(deltas[["x", "y"]], axis=1)
  sns.histplot(distances, ax=ax, bins=np.arange(20))

  plt.subplot(gs[6])
  from tqdm import tqdm
  from scipy.stats.kde import gaussian_kde

  # make these smaller to increase the resolution
  dx, dy = 2, 2

  # generate 2 2d grids for the x & y bounds
  w = reader.width
  h = reader.height
  result = {}

  # Get the centroid of the bodypart by averaging the bodypart positions in each frame
  data = bodypart_df[['y', 'x']].dropna().values.T

  # Get the density of time spent in location (down sampled for 1 frame every 100)
  k = gaussian_kde(data[:,::1], )
  mgrid = np.mgrid[:h, :w]
  z = k(mgrid.reshape(2, -1))


  plt.title("Box Reference")
  plt.imshow(reader[0])

  plt.title("Left Hand")
  plt.imshow(frame) # plotting a frame sets up the matplotlib axis correctly
  plt.imshow(z.reshape(mgrid.shape[1:]))
  plt.contourf(z.reshape(mgrid.shape[1:]), cmap='seismic', alpha=1, levels=20)

  plt.show()